# === 1. Data Loading and Exploration ===

In [ ]:
# === 1. Data Loading and Exploration ===

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, silhouette_score
from scipy.stats import zscore

# Load the dataset
csv_file = "data/eco2mix.csv"
df = pd.read_csv(csv_file)

# Convert 'Date - Time' to datetime format
df['Date - Time'] = pd.to_datetime(df['Date - Time'])

# Inspect the dataset
print("Dataset Overview:")
print(df.info())
print("\nFirst Few Rows:")
print(df.head())

## **Analysis:**  
 - This dataset contains detailed information about energy production and consumption across different regions and energy sources.  
 - Key columns include `Consumption (MW)`, `Production (MW)`, `Wind (MW)`, `Solar (MW)`, `Hydraulic (MW)`, and timestamps (`Date - Time`).  
 - Our goal is to analyze, cluster, and predict trends while exploring the dataset comprehensively.

# === 3. Clustering with Optimal Cluster Selection ===

In [ ]:
# === 3. Clustering with Optimal Cluster Selection ===


# Use a subset of data (1%) for efficient calculation of optimal clusters
subset = df.sample(frac=0.01, random_state=42)
subset_metrics = subset[['Consumption (MW)', 'Production (MW)', 'Wind (MW)', 'Solar (MW)', 'Hydraulic (MW)']].dropna()

# Scale the subset data
scaler = StandardScaler()
scaled_subset_metrics = scaler.fit_transform(subset_metrics)

# Determine the optimal number of clusters using Elbow Method and Silhouette Scores
inertia = []
silhouette_scores = []
range_n_clusters = range(2, 11)

for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_subset_metrics)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(scaled_subset_metrics, cluster_labels))

# Plot Elbow Curve
plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, inertia, marker='o', linestyle='--')
plt.title("Elbow Method for Optimal Clusters (Subset)", fontsize=16)
plt.xlabel("Number of Clusters", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.grid(alpha=0.3)
plt.show()

# Plot Silhouette Scores
plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, silhouette_scores, marker='o', linestyle='--', color='orange')
plt.title("Silhouette Scores for Optimal Clusters (Subset)", fontsize=16)
plt.xlabel("Number of Clusters", fontsize=14)
plt.ylabel("Silhouette Score", fontsize=14)
plt.grid(alpha=0.3)
plt.show()

# Select the optimal number of clusters based on silhouette scores
optimal_clusters = list(range_n_clusters)[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters based on subset: {optimal_clusters}")

# Apply optimal clustering to the full dataset
full_metrics = df[['Consumption (MW)', 'Production (MW)', 'Wind (MW)', 'Solar (MW)', 'Hydraulic (MW)']].dropna()
scaled_full_metrics = scaler.transform(full_metrics)

# Apply clustering to the subset
kmeans_subset = KMeans(n_clusters=optimal_clusters, random_state=42)
subset['Cluster'] = kmeans_subset.fit_predict(scaled_subset_metrics)

# Visualize clustering for the 1% subset
plt.figure(figsize=(10, 6))
markers = ['o', 's', 'D', '^', 'P', '*', 'X']  # Define different marker styles

for cluster in range(optimal_clusters):
    cluster_data = subset[subset['Cluster'] == cluster]
    plt.scatter(
        cluster_data['Production (MW)'], 
        cluster_data['Consumption (MW)'], 
        label=f'Cluster {cluster}', 
        alpha=0.7, 
        marker=markers[cluster % len(markers)]  # Cycle through markers
    )

plt.title("Clustering of Regions (1% Subset Data)", fontsize=16)
plt.xlabel("Production (MW)", fontsize=14)
plt.ylabel("Consumption (MW)", fontsize=14)
plt.legend(title="Cluster")
plt.grid(alpha=0.3)
plt.show()

## **Analysis:**  
 - Using a subset of the data ensures efficient calculation of the optimal cluster number.  
 - The Elbow Method and Silhouette Scores indicate the best k-value for clustering.  
 - Applying the determined number of clusters to the full dataset groups regions effectively.  
 - The scatterplot shows distinct clusters, providing actionable insights into energy production and consumption patterns.  

# === 4. Outlier Detection ===

In [ ]:
# === 4. Outlier Detection ===

# Compute Z-scores
df['Production_Zscore'] = zscore(df['Production (MW)'])
df['Consumption_Zscore'] = zscore(df['Consumption (MW)'])

# Identify and Analyze Outliers
production_outliers = df[np.abs(df['Production_Zscore']) > 3]
consumption_outliers = df[np.abs(df['Consumption_Zscore']) > 3]

print(f"Number of production outliers: {len(production_outliers)}")
print(f"Number of consumption outliers: {len(consumption_outliers)}")

# Visualize Outliers in Production and Consumption
fig = px.scatter(df, x='Production (MW)', y='Consumption (MW)', 
                 color=np.abs(df['Production_Zscore']) > 3, 
                 title="Outlier Detection in Production and Consumption",
                 labels={"color": "Production Outlier"})
fig.show()

# Explore Outliers for Specific Regions
top_outlier_regions = production_outliers['Region'].value_counts().head()
print("\nTop Regions with Production Outliers:\n", top_outlier_regions)

# Visualize Outlier Trends Over Time
fig = px.line(production_outliers, x='Date - Time', y='Production (MW)', color='Region',
              title="Production Outliers Over Time by Region",
              labels={"Region": "Outlier Region"})
fig.show()

fig = px.line(consumption_outliers, x='Date - Time', y='Consumption (MW)', color='Region',
              title="Consumption Outliers Over Time by Region",
              labels={"Region": "Outlier Region"})
fig.show()

# **Analysis:**  
 - Significant outliers in production and consumption highlight rare events or anomalies.  
 - Specific regions dominate outlier counts, suggesting potential localized issues or unique conditions.  
 - Time-series visualizations of outliers provide insights into the timing and frequency of these anomalies.  

# === 5. Optimized Prediction ===

In [ ]:
# === 5. Optimized Prediction  ===

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np

# Ensure the 'Month' and 'Hour' columns are extracted
df['Month'] = pd.to_datetime(df['Date - Time']).dt.month  # Extract month
df['Hour'] = pd.to_datetime(df['Date - Time']).dt.hour   # Extract hour

# Select relevant features based on correlation
features = ['Production (MW)', 'Hydraulic (MW)', 'Month', 'Hour']
target = 'Consumption (MW)'

# Prepare the dataset
X = df.dropna(subset=features + [target])[features]
y = df.dropna(subset=features + [target])[target]

# Train-test split (80% Train, 20% Test)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LightGBM Model
lgb_model = lgb.LGBMRegressor(n_estimators=200, max_depth=15, learning_rate=0.1, num_leaves=100, random_state=42, n_jobs=-1)
lgb_model.fit(train_X, train_y)

# Predict and Evaluate
lgb_pred_y = lgb_model.predict(test_X)
lgb_mse = mean_squared_error(test_y, lgb_pred_y)
lgb_rmse = np.sqrt(lgb_mse)
lgb_mae = mean_absolute_error(test_y, lgb_pred_y)

print(f"LightGBM MSE: {int(lgb_mse)}")
print(f"LightGBM RMSE: {int(lgb_rmse)}")
print(f"LightGBM MAE: {int(lgb_mae)}")

# Visualize Predictions (Actual vs Predicted)
fig = px.scatter(x=test_y, y=lgb_pred_y, 
                 title="Actual vs Predicted Consumption (LightGBM)",
                 labels={"x": "Actual Consumption", "y": "Predicted Consumption"})
fig.update_traces(marker=dict(size=5, opacity=0.7))
fig.show()

# Feature Importance
importance_df = pd.DataFrame({'Feature': features, 'Importance': lgb_model.feature_importances_})
fig = px.bar(importance_df.sort_values(by="Importance", ascending=False), 
             x='Feature', y='Importance', 
             title="Feature Importance in LightGBM Model")
fig.show()

## **Analysis:**  
 - LightGBM provides an efficient solution for large datasets, balancing speed and accuracy.  
 - The feature importance chart highlights key variables influencing consumption predictions.  
 Best Parameters for LightGBM: {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 200, 'num_leaves': 100}

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# === Enhanced Visualization ===

# 1. Scatter Plot: Actual vs Predicted with Ideal Line
fig = go.Figure()

# Add actual vs predicted scatter points
fig.add_trace(go.Scatter(
    x=test_y, 
    y=lgb_pred_y, 
    mode='markers', 
    name='Predictions',
    marker=dict(size=5, opacity=0.7)
))

# Add perfect line (y=x)
fig.add_trace(go.Scatter(
    x=[test_y.min(), test_y.max()],
    y=[test_y.min(), test_y.max()],
    mode='lines',
    name='Ideal Line',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title="Actual vs Predicted Consumption (LightGBM)",
    xaxis_title="Actual Consumption (MW)",
    yaxis_title="Predicted Consumption (MW)",
    legend_title="Legend",
    width=800, height=600
)
fig.show()

# 2. Residual Plot
residuals = test_y - lgb_pred_y
fig = px.scatter(
    x=lgb_pred_y, 
    y=residuals, 
    labels={'x': 'Predicted Consumption (MW)', 'y': 'Residuals (MW)'},
    title='Residual Plot (LightGBM Predictions)'
)
fig.update_traces(marker=dict(size=5, opacity=0.7))
fig.add_hline(y=0, line_dash="dash", line_color="red")  # Add a horizontal line at residual=0
fig.show()

# 3. Distribution Plot: Actual vs Predicted
fig = go.Figure()

# Add distribution for actual values
fig.add_trace(go.Histogram(
    x=test_y, 
    name='Actual', 
    opacity=0.6, 
    marker_color='blue'
))

# Add distribution for predicted values
fig.add_trace(go.Histogram(
    x=lgb_pred_y, 
    name='Predicted', 
    opacity=0.6, 
    marker_color='orange'
))

fig.update_layout(
    title="Distribution of Actual vs Predicted Consumption",
    barmode='overlay',
    xaxis_title="Consumption (MW)",
    yaxis_title="Frequency",
    legend_title="Legend",
    width=800, height=600
)
fig.show()

# 4. Feature Importance (Horizontal Bar Chart)
importance_df_sorted = importance_df.sort_values(by="Importance", ascending=True)

fig = px.bar(
    importance_df_sorted, 
    x='Importance', 
    y='Feature', 
    orientation='h', 
    title="Feature Importance (LightGBM)"
)
fig.update_layout(
    xaxis_title="Importance Score",
    yaxis_title="Features",
    width=800, height=600
)
fig.show()

# Residuals Heatmap by Hour and Month
df_test = df.loc[test_X.index].copy()
df_test['Residuals'] = residuals

heatmap_data = df_test.pivot_table(
    values='Residuals', 
    index='Hour', 
    columns='Month', 
    aggfunc='mean'
)

fig = px.imshow(
    heatmap_data, 
    labels={'color': 'Residuals (MW)', 'x': 'Month', 'y': 'Hour'},
    title="Average Residuals by Hour and Month",
    color_continuous_scale='RdBu_r'
)
fig.update_layout(width=800, height=600)
fig.show()

# Time-Series of Actual vs Predicted
test_y_index = test_y.reset_index(drop=True)
fig = go.Figure()

# Add actual values
fig.add_trace(go.Scatter(
    x=test_y_index.index, 
    y=test_y_index, 
    mode='lines', 
    name='Actual', 
    line=dict(color='blue')
))

# Add predicted values
fig.add_trace(go.Scatter(
    x=test_y_index.index, 
    y=lgb_pred_y, 
    mode='lines', 
    name='Predicted', 
    line=dict(color='orange')
))

fig.update_layout(
    title="Time-Series of Actual vs Predicted Consumption",
    xaxis_title="Index (Time Ordered)",
    yaxis_title="Consumption (MW)",
    legend_title="Legend",
    width=800, height=600
)
fig.show()

# **Analysis:**  
- Strong agreement between actual and predicted consumption values. 
- Bar chart highlights the key predictors influencing energy consumption.
- Time series comparison of actual vs predicted consumption reveals systematic trends and residual errors.

# === OPTIONAL: Best Parameters for LightBGM ===

In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

# Define a parameter grid for LightGBM
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'max_depth': [5, 10, 15]
}

# Create the LightGBM model
lgb_model = lgb.LGBMRegressor(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(train_X, train_y)

# Get the best parameters and train the model
best_params = grid_search.best_params_
print(f"Best Parameters for LightGBM: {best_params}")

# Train the model with the best parameters
optimized_lgb_model = lgb.LGBMRegressor(**best_params, random_state=42)
optimized_lgb_model.fit(train_X, train_y)

# Predict and evaluate the optimized model
optimized_pred_y = optimized_lgb_model.predict(test_X)
optimized_mse = mean_squared_error(test_y, optimized_pred_y)
optimized_rmse = np.sqrt(optimized_mse)
optimized_mae = mean_absolute_error(test_y, optimized_pred_y)

print(f"Optimized LightGBM MSE: {int(optimized_mse)}")
print(f"Optimized LightGBM RMSE: {int(optimized_rmse)}")
print(f"Optimized LightGBM MAE: {int(optimized_mae)}")